In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sqlite3
import pandas as pd
import os

def inspect_all_sqlite_dbs(folder_path):
    db_files = [f for f in os.listdir(folder_path) if f.endswith(('.db', '.sqlite'))]

    if not db_files:
        print("Aucun fichier .db ou .sqlite trouvé dans le dossier.")
        return

    for db_file in db_files:
        db_path = os.path.join(folder_path, db_file)
        print(f"\n{'='*60}\nInspection de: {db_file}\n{'='*60}\n")

        try:
            conn = sqlite3.connect(db_path)
            cursor = conn.cursor()

            cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            tables = cursor.fetchall()

            if not tables:
                print("Aucune table trouvée dans cette base.")
                continue

            print(f"Tables: {', '.join([t[0] for t in tables])}\n")

            for table in tables:
                table_name = table[0]
                print(f"--- Table: {table_name} ---")

                cursor.execute(f"PRAGMA table_info({table_name});")
                columns = cursor.fetchall()
                print("Colonnes:")
                for col in columns:
                    print(f"  - {col[1]} ({col[2]})")

                try:
                    df = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 5;", conn)
                    print("\nAperçu des données (5 premières lignes):")
                    print(df)
                    print("\n" + "-"*60 + "\n")
                except Exception as e:
                    print(f"\nErreur lors de la lecture des données: {e}\n")

            conn.close()
        except Exception as e:
            print(f"Erreur lors de l'ouverture de {db_file}: {e}")

# Chemin vers ton dossier dans Colab
inspect_all_sqlite_dbs("/content/drive/MyDrive/Projets/FDD/bd/")


Inspection de: ieee_llm_articles.db

Tables: articles, sqlite_sequence, authors, keywords

--- Table: articles ---
Colonnes:
  - id (INTEGER)
  - title (TEXT)
  - link (TEXT)
  - abstract (TEXT)
  - doi (TEXT)
  - date_of_publication (TEXT)
  - publisher (TEXT)
  - published_in (TEXT)
  - print_issn (TEXT)
  - electronic_issn (TEXT)

Aperçu des données (5 premières lignes):
   id                                              title  \
0   1  Cost-Effective Extension of DRAM-PIM for Group...   
1   2  Maestro: LLM-Driven Collaborative Automation o...   
2   3  Achieving Peak Performance for Large Language ...   
3   4  LLM-DSK: A Domain-Specific Semantic Knowledge-...   
4   5  A Comprehensive Survey on LLM-Powered Recommen...   

                                             link  \
0  https://ieeexplore.ieee.org/document/10886951/   
1  https://ieeexplore.ieee.org/document/10758700/   
2  https://ieeexplore.ieee.org/document/10589417/   
3  https://ieeexplore.ieee.org/document/11083757/